In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
import joblib
import seaborn as sns
import matplotlib.pyplot as plt


In [ ]:
# Load the dataset
data = pd.read_csv('Genre-Classification-Dataset/train_data.txt', sep=' ::: ', engine='python', header=None, names=['id', 'title', 'genre', 'plot'])

# Filter out genres with less than 5 instances (optional, to avoid overfitting on rare genres)
genre_counts = data['genre'].value_counts()
valid_genres = genre_counts[genre_counts >= 5].index
data = data[data['genre'].isin(valid_genres)]

# Split the data into features and labels
X = data['plot']  # Movie plot descriptions
y = data['genre']  # Movie genres


In [ ]:
# Split the data into training and testing sets (80% train, 20% test)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)


In [ ]:
# Using TF-IDF Vectorizer with n-grams (unigrams and bigrams)
vectorizer = TfidfVectorizer(ngram_range=(1, 2), max_features=10000)
X_train_tfidf = vectorizer.fit_transform(X_train)
X_test_tfidf = vectorizer.transform(X_test)


In [ ]:
# Logistic Regression model with cross-validation and hyperparameter tuning
logreg = LogisticRegression()

# Define parameter grid for tuning
param_grid = {
    'C': [0.1, 1, 10],  # Regularization strength
    'penalty': ['l2'],  # Use L2 regularization
    'solver': ['liblinear', 'lbfgs']  # Solvers
}

# Use GridSearchCV to find the best hyperparameters
grid_search = GridSearchCV(logreg, param_grid, cv=5, scoring='accuracy', verbose=1, n_jobs=-1)
grid_search.fit(X_train_tfidf, y_train)

# Get the best estimator (best model)
best_model = grid_search.best_estimator_

# Train the final Logistic Regression model with the best parameters
best_model.fit(X_train_tfidf, y_train)


In [ ]:
# Make predictions on the test set
y_pred = best_model.predict(X_test_tfidf)

# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy * 100:.2f}%")

# Confusion matrix
conf_matrix = confusion_matrix(y_test, y_pred)
plt.figure(figsize=(10, 8))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', xticklabels=best_model.classes_, yticklabels=best_model.classes_)
plt.title('Confusion Matrix')
plt.xlabel('Predicted Genre')
plt.ylabel('True Genre')
plt.show()

# Classification report
print("Classification Report:\n")
print(classification_report(y_test, y_pred))


In [ ]:
# Save the trained model and vectorizer for later use in prediction
joblib.dump(best_model, 'random_forest_model.pkl')
joblib.dump(vectorizer, 'tfidf_vectorizer.pkl')

print("Model and TF-IDF vectorizer saved successfully.")
